In [2]:
# New Jupyter Notebook: Model Development with Tissuenet

# This notebook demonstrates how to use the Tissuenet dataset to train a deep learning model
# and run it seamlessly on Google Colab.

# 1. Environment Setup
!pip install -q tensorflow scikit-learn

# 2. Import Required Libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import f1_score
import numpy as np
import matplotlib.pyplot as plt

# 3. Download and Preprocess the Tissuenet Dataset
# Download Data
import urllib.request
import os

data_url = "https://deepcell-data.s3-us-west-1.amazonaws.com/demos/tissuenet_v1.0/tissuenet_v1.0.npz"
data_path = "tissuenet_v1.0.npz"
if not os.path.exists(data_path):
    try:
        urllib.request.urlretrieve(data_url, data_path)
        print("Dataset downloaded successfully.")
    except Exception as e:
        print(f"Error occurred while downloading the dataset: {e}")
        raise

# Load Data
with np.load(data_path) as f:
    X_train, y_train = f['X_train'], f['y_train']
    X_test, y_test = f['X_test'], f['y_test']

print(f"Training data shape: {X_train.shape}, Labels: {y_train.shape}")
print(f"Test data shape: {X_test.shape}, Labels: {y_test.shape}")

# Normalize the data
X_train = X_train / 255.0
X_test = X_test / 255.0

# 4. Define Model Architectures
# Model 1: Basic CNN
model_1 = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=X_train.shape[1:]),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model_1.compile(optimizer=Adam(learning_rate=0.001),
                loss='binary_crossentropy',
                metrics=['accuracy'])

# Model 2: Deeper CNN
model_2 = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=X_train.shape[1:]),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model_2.compile(optimizer=Adam(learning_rate=0.001),
                loss='binary_crossentropy',
                metrics=['accuracy'])

# 5. Train Models
history_1 = model_1.fit(X_train, y_train, validation_split=0.2, epochs=10, batch_size=32, verbose=1)
history_2 = model_2.fit(X_train, y_train, validation_split=0.2, epochs=10, batch_size=32, verbose=1)

# 6. Evaluate Models
# Evaluate Model 1
predictions_1 = (model_1.predict(X_test) > 0.5).astype(np.uint8)
f1_1 = f1_score(y_test.flatten(), predictions_1.flatten(), average='binary')
print(f"F1 Score for Model 1: {f1_1:.4f}")

# Evaluate Model 2
predictions_2 = (model_2.predict(X_test) > 0.5).astype(np.uint8)
f1_2 = f1_score(y_test.flatten(), predictions_2.flatten(), average='binary')
print(f"F1 Score for Model 2: {f1_2:.4f}")

# Choose the Best Model
best_model = model_1 if f1_1 > f1_2 else model_2
print(f"The best model is: {'Model 1' if f1_1 > f1_2 else 'Model 2'}")

# 7. Save the Best Model
best_model.save('best_tissuenet_model.h5')

# 8. Visualize Results
plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.imshow(X_test[0, ..., 0], cmap='gray')
plt.title("Input")

plt.subplot(1, 3, 2)
plt.imshow(y_test[0, ..., 0], cmap='gray')
plt.title("Ground Truth")

plt.subplot(1, 3, 3)
plt.imshow(predictions_1[0, ..., 0] if f1_1 > f1_2 else predictions_2[0, ..., 0], cmap='gray')
plt.title("Prediction")

plt.show()

# This notebook demonstrates how to use the Tissuenet dataset and evaluates different CNN architectures to find the best model based on F1 score.


HTTPError: HTTP Error 403: Forbidden